# Applied Competitive Lab In Data Science

By: Micha Hashkes, Tamar Ben-Ami, Noa Bitan

## Stage 1 - Reading DataFrame from fires DB

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
import numpy as np
# import rasterio
import requests
import sqlite3
from config import *
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from utils import *

## Stage 3 - Pipeline Activation Testing

In [9]:
df = create_dataframe()
X_train, X_test, y_train, y_test = train_test_split(df.drop(LABEL_FIELD, axis=1), df[LABEL_FIELD], test_size=0.3)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [10]:
# train_df = train_df.iloc[:100]
# test_df = test_df.iloc[:10]

In [ ]:
train_gdf, test_gdf = df_to_gdf(train_df), df_to_gdf(test_df)
train_gdf, train_features = extract_features_train(train_gdf)
test_gdf, test_features = extract_features_test(test_gdf, train_gdf)

electricity_transition_lines
electricity_power_plants
indian_reserves
military_bases
national_parks
transportation_amtrak_stations
transportation_rail_network_lines
transportation_rail_network_nodes
transportation_railroad_bridges
transportation_railroad_grade_crossings
water_bodies
urban_areas
electricity_transition_lines
electricity_power_plants
indian_reserves
military_bases
national_parks


In [ ]:
train_gdf[train_features]

In [ ]:
model = fit_model(train_gdf[train_features], train_gdf[LABEL_FIELD])

In [ ]:
y_pred = predict_results(test_gdf[train_features], model)
print(classification_report(test_gdf[LABEL_FIELD], y_pred))

In [9]:
plot_feature_importance(model)

AttributeError: 'RandomForestClassifier' object has no attribute 'feature_names_in_'

## Stage 4 - Pipeline Activation

#### In order to use our model, you'll need:
1. put your to_test dataframe in to_test_df variable
2. run "run me" cell which will create the trained model
3. use "predict_results" function who takes test_df and a model

In [7]:
# fill me
train_df = None
test_df = None

In [8]:
train_gdf, test_gdf = df_to_gdf(train_df), df_to_gdf(test_df)

AttributeError: 'NoneType' object has no attribute 'LONGITUDE'

In [ ]:
train_features, test_features = extract_features_train(train_gdf),\
                                extract_features_test(test_gdf, train_gdf)

In [ ]:
model = fit_model(train_gdf[train_features], train_gdf[LABEL_FIELD])
y_pred = predict_results(test_gdf[train_features], model)